In [0]:
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import input_file_name, count, when, col, lit

# 1. Definir o esquema manualmente
json_schema = StructType([
    StructField("codigo", StringType(), True),
    StructField("data", StringType(), True),
    StructField("desconto", StringType(), True),
    StructField("link", StringType(), True),
    StructField("nome", StringType(), True),
    StructField("parcelamento", StringType(), True),
    StructField("preco", StringType(), True)
])

# Caminho para a pasta que contém os arquivos JSON
json_folder_path = "/Volumes/nintendodatabricks037cbq_workspace/nintendo/inbound/*.json"


print(f"Tentando ler arquivos JSON de: {json_folder_path}")

try:
    # --- Leitura dos arquivos JSON ---
    # Adicionando a opção pathGlobFilter para garantir que apenas .json sejam lidos explicitamente
    df_json_unificado = spark.read \
                             .option("multiLine", "true") \
                             .option("mode", "PERMISSIVE") \
                             .option("columnNameOfCorruptRecord", "_corrupt_record") \
                             .schema(json_schema) \
                             .json(json_folder_path) \
                             .withColumn("file_source", input_file_name()) # Adiciona coluna com o caminho do arquivo

    print("\n--- Esquema do DataFrame unificado ---")
    df_json_unificado.printSchema()

    # --- 1. Verificação de Arquivos Lidos vs. Arquivos Esperados ---
    # Contar quantos arquivos foram realmente processados pelo Spark
    files_processed_df = df_json_unificado.select("file_source").distinct().count()
    processed_file_paths = [row.file_source for row in df_json_unificado.select("file_source").distinct().collect()]

    # Para obter o número de arquivos esperados diretamente de um Volume,
    # a melhor prática é usar a API do Spark para listar os arquivos, já que ela entende os caminhos de Volumes.
    try:
        # Pega a parte da pasta do json_folder_path (removendo "/*.json")
        base_folder_path = json_folder_path.replace("/*.json", "")
        # Lista os arquivos .json na pasta usando Spark para compatibilidade com Volumes
        # Isso cria um DataFrame com o caminho de cada arquivo
        expected_json_files_df = spark.read.format("binaryFile") \
                                            .option("pathGlobFilter", "*.json") \
                                            .load(base_folder_path) \
                                            .select("path")

        num_expected_files = expected_json_files_df.count()
        expected_file_paths_spark = [row.path for row in expected_json_files_df.collect()]

    except Exception as e_list:
        print(f"Não foi possível listar arquivos usando Spark no caminho '{base_folder_path}': {e_list}")
        print("Prosseguindo com a contagem apenas dos arquivos processados pelo DataFrame.")
        num_expected_files = -1 # Indica que não foi possível obter o número esperado
        expected_file_paths_spark = []


    print(f"\n--- Verificação de Arquivos ---")
    if num_expected_files != -1:
        print(f"Número de arquivos .json esperados na pasta '{base_folder_path}': {num_expected_files}")
    else:
        print(f"Não foi possível determinar o número esperado de arquivos .json na pasta '{base_folder_path}'.")
    print(f"Número de arquivos únicos processados no DataFrame: {files_processed_df}")

    if num_expected_files != -1 and files_processed_df == num_expected_files:
        print("STATUS: OK - Todos os arquivos JSON esperados foram processados.")
    elif num_expected_files != -1 and files_processed_df != num_expected_files:
        print("STATUS: ALERTA - O número de arquivos processados difere do esperado. Investigue!")
        # Para identificar quais arquivos podem ter sido ignorados:
        missing_files = [path for path in expected_file_paths_spark if path not in processed_file_paths]
        if missing_files:
            print(f"Arquivos JSON esperados que não foram encontrados no DataFrame: {missing_files}")
    else:
        print("STATUS: INCONCLUSIVO - Não foi possível comparar o número de arquivos esperados com os processados.")

    # --- 2. Contagem de Registros ---
    total_records_loaded = df_json_unificado.count()
    print(f"\n--- Verificação de Registros ---")
    print(f"Total de registros carregados no DataFrame: {total_records_loaded}")

    if total_records_loaded == 0 and num_expected_files > 0:
        print("STATUS: ALERTA - Nenhum registro foi carregado, mas há arquivos JSON esperados. Verifique se os arquivos estão vazios ou malformados.")
    elif total_records_loaded > 0:
        print("STATUS: OK - Registros foram carregados com sucesso.")
    else:
        print("STATUS: N/A - Nenhuma expectativa de registros (pasta vazia ou sem arquivos .json).")

    # --- 3. Validação Básica de Dados (Verificar nulos em colunas críticas) ---
    print(f"\n--- Verificação de Nulos em Colunas Críticas ---")
    critical_columns = ["codigo", "nome", "preco"] # Exemplo

    for col_name in critical_columns:
        null_count = df_json_unificado.filter(col(col_name).isNull()).count()
        if null_count > 0:
            print(f"ALERTA: Coluna '{col_name}' possui {null_count} valores nulos.")
        else:
            print(f"OK: Coluna '{col_name}' não possui valores nulos.")

    # --- 4. Verificação de Registros Corrompidos (se houver a coluna _corrupt_record) ---
    if "_corrupt_record" in df_json_unificado.columns:
        corrupt_records_count = df_json_unificado.filter(col("_corrupt_record").isNotNull()).count()
        if corrupt_records_count > 0:
            print(f"\nALERTA: Encontrados {corrupt_records_count} registros corrompidos no _corrupt_record. Investigue!")
            df_json_unificado.filter(col("_corrupt_record").isNotNull()).select("file_source", "_corrupt_record").show(truncate=False)
        else:
            print("\nOK: Não foram encontrados registros corrompidos na coluna _corrupt_record.")

    # --- 5. Dropar a coluna 'file_source' ---
    # É importante dropar a coluna *depois* de todas as verificações que a utilizam
    df_json_unificado = df_json_unificado.drop("file_source")
    print("\n--- Coluna 'file_source' foi removida do DataFrame final. ---")
    print("\nEsquema do DataFrame final:")
    df_json_unificado.printSchema()
    print("\nPrimeiras 5 linhas do DataFrame final:")
    df_json_unificado.show(5, truncate=False)


except Exception as e:
    print(f"Ocorreu um erro durante a leitura ou verificação de qualidade: {e}")

In [0]:
# Certifique-se de que df_json_unificado esteja disponível após as etapas anteriores

# Caminho de destino para a tabela Delta
delta_table_path = "/Volumes/nintendodatabricks037cbq_workspace/nintendo/bronze"

print(f"Iniciando o salvamento do DataFrame no formato Delta em: {delta_table_path}")

try:
    # --- Passo 1: Obter a contagem de linhas ANTES de salvar ---
    num_rows_to_save = df_json_unificado.count()
    print(f"Número de linhas no DataFrame a ser salvo: {num_rows_to_save}")

    # --- Passo 2: Salvar o DataFrame no formato Delta ---
    df_json_unificado.write \
                     .format("delta") \
                     .mode("overwrite") \
                     .partitionBy("data") \
                     .save(delta_table_path)

    print(f"DataFrame salvo com sucesso como tabela Delta particionada por 'data' em: {delta_table_path}")

    # --- Início das Verificações de Qualidade Pós-Gravação ---

    # --- 1. Garantir que os dados foram salvos no caminho ---
    print(f"\n--- Verificação: Leitura da Tabela Delta Salva ---")
    df_delta_read = spark.read.format("delta").load(delta_table_path)
    print("Esquema da tabela Delta lida:")
    df_delta_read.printSchema()
    print("Primeiras 5 linhas da tabela Delta lida:")
    df_delta_read.show(5, truncate=False)

    if df_delta_read.isEmpty():
        print(f"ALERTA: A tabela Delta salva em '{delta_table_path}' está vazia ou não pôde ser lida.")
    else:
        print(f"OK: A tabela Delta foi lida com sucesso de '{delta_table_path}'.")


    # --- 2. Verificar se a quantidade de linhas salvas condiz com o que está salvo ---
    num_rows_saved = df_delta_read.count()
    print(f"\n--- Verificação: Contagem de Linhas Salvas ---")
    print(f"Número de linhas salvas na tabela Delta: {num_rows_saved}")

    if num_rows_saved == num_rows_to_save:
        print(f"STATUS: OK - A quantidade de linhas salvas ({num_rows_saved}) corresponde à quantidade de linhas no DataFrame original ({num_rows_to_save}).")
    else:
        print(f"ALERTA: A quantidade de linhas salvas ({num_rows_saved}) NÃO CORRESPONDE à quantidade de linhas no DataFrame original ({num_rows_to_save}). Investigue!")


    # --- 3. Verificar se realmente foi particionado ---
    print(f"\n--- Verificação: Particionamento por 'data' ---")

    # Substituindo %fs ls -l por dbutils.fs.ls()
    print("Conteúdo do diretório Delta (buscando por pastas de partição usando dbutils):")
    try:
        # Lista os subdiretórios no caminho Delta. Esperamos ver pastas como data=YYYY-MM-DD
        delta_contents = dbutils.fs.ls(delta_table_path)
        partition_folders_found = [f.name for f in delta_contents if f.isDir and "=" in f.name]
        if partition_folders_found:
            print(f"Pastas de partição detectadas (ex: {', '.join(partition_folders_found[:3])}...):")
            # Opcional: mostrar todas as pastas de partição se for um número pequeno
            # for p_folder in partition_folders_found:
            #     print(f"  - {p_folder}")
        else:
            print("Nenhuma pasta de partição padrão (ex: 'data=...') detectada diretamente no caminho raiz.")

    except Exception as ls_e:
        print(f"ALERTA: Erro ao listar conteúdo do diretório Delta com dbutils.fs.ls(): {ls_e}")


    # O método mais confiável continua sendo usar o DESCRIBE DETAIL
    try:
        spark.sql(f"DESCRIBE DETAIL delta.`{delta_table_path}`").show(truncate=False)
        table_details_df = spark.sql(f"DESCRIBE DETAIL delta.`{delta_table_path}`")
        partition_columns = table_details_df.select("partitionColumns").collect()[0][0] # Pega o primeiro elemento da lista

        if "data" in partition_columns:
            print(f"STATUS: OK - A tabela Delta está particionada pela coluna 'data'.")
        else:
            print(f"ALERTA: A tabela Delta NÃO parece estar particionada pela coluna 'data'. Partições encontradas: {partition_columns}")

    except Exception as sql_e:
        print(f"ALERTA: Não foi possível obter detalhes da tabela Delta (verifique o log): {sql_e}")
        # A linha abaixo não pode ser executada por estar dentro de um try-except, por isso a retirei
        # print("Tente verificar manualmente a estrutura de pastas com '%fs ls -l dbfs:/nintendo/bronze/'.")


except Exception as e:
    print(f"Ocorreu um erro geral ao salvar ou verificar a tabela Delta: {e}")